In [387]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [388]:
critic_name= []
critic_url= []
for letter in 'abcdefghijklmnopqrstuvwxyz':
    critics_url = "https://www.rottentomatoes.com/critics/authors?letter="+letter
    get_critics = requests.get(critics_url)
    critics_soup = BeautifulSoup(get_critics.text,"html.parser")
    for i in critics_soup.find_all('li', 'critics__list-item'):
        critic_name.append(i.find("a").string)
        critic_url.append("https://www.rottentomatoes.com"+str(i.find("a")['href']))

In [389]:
critics_info = { "Critic Name": critic_name,
                "URL": critic_url
}
df_3 = pd.DataFrame(critics_info)

In [390]:
df_3

,Critic Name,URL
0,Josefine A.,https://www.rottentomatoes.com/critic/josefine-a
1,Lysalex Hernández A.,https://www.rottentomatoes.com/critic/lysalex-hernandez-a
2,Alex Abad-Santos,https://www.rottentomatoes.com/critic/alex-abad-santos
3,Nicole Abadee,https://www.rottentomatoes.com/critic/nicole-abadee
4,Davide Abbatescianni,https://www.rottentomatoes.com/critic/davide-abbatescianni
...,...,...
3848,Esther Zuckerman,https://www.rottentomatoes.com/critic/esther-zuckerman
3849,Lauren Zupkus,https://www.rottentomatoes.com/critic/lauren-zupkus
3850,David Zurawik,https://www.rottentomatoes.com/critic/david-zurawik
3851,Janire Zurbano,https://www.rottentomatoes.com/critic/janire-zurbano


In [190]:
def Critic_scrapper(URL,Name):
    #init_df_table
    critics_url_2 = URL+"/movies?page=1"
    print(critics_url_2)
    get_critics_2 = requests.get(critics_url_2)
    critics_soup_2 = BeautifulSoup(get_critics_2.text,"html.parser")
    No_Review_tag = critics_soup_2.find_all("section",{"class":"panel panel-rt panel-box"})
    if "No reviews available." in str(No_Review_tag):
        pass
    else:
        Movie_tag =  critics_soup_2.find_all("h2",{"class":"panel-heading js-review-type"})[0].string
        if "Movie" in Movie_tag:
            page =critics_soup_2.find('table', {'class': "table table-striped critic-review-table responsive-table"})
            movie_url_critics = []
            for i in page.find_all("td",{"class":"col-xs-12 col-sm-6 critic-review-table__title-column"}):
                try:
                    movie_url_critics.append(i.find("a")['href'])
                except:
                    movie_url_critics.append("No data")
                ####check
            df_5 = pd.read_html(str(page))[0]
            df_5['Movie URL'] = movie_url_critics
            df_5['Critic Name'] = Name
            df_5
    #         print("Page 1 done"+Name)
            #INIT DF for current critics
            tmp_tag = 0
            count=2
            while tmp_tag != -1:
                critics_url_2 = URL+"/movies?page="+str(count)
                #print(critics_url_2)
                get_critics_2 = requests.get(critics_url_2)
                critics_soup_2 = BeautifulSoup(get_critics_2.text,"html.parser")
                page =critics_soup_2.find('table', {'class': "table table-striped critic-review-table responsive-table"})
                tmp_checker =len(page.find_all("td",{"class":"col-xs-12 col-sm-6 critic-review-table__title-column"}))
                #print(tmp_checker)
                if tmp_checker==0:
                    tmp_tag = -1
                    break
                movie_url_critics = []
                #print(page.find_all("td",{"class":"col-xs-12 col-sm-6 critic-review-table__title-column"}))
                for i in page.find_all("td",{"class":"col-xs-12 col-sm-6 critic-review-table__title-column"}):
                    #print(i)
                    try:
                        movie_url_critics.append(i.find("a")['href'])
                    except:
                        movie_url_critics.append("No data")
                df_5_tmp = pd.read_html(str(page))[0]
                df_5_tmp['Movie URL'] = movie_url_critics
                df_5_tmp['Critic Name'] = Name
                df_5=df_5.append(df_5_tmp)
    #             print("Page "+str(count)+" done"+Name)
                count+=1
            return df_5

In [ ]:
#init_df_table
critics_url_2 = "https://www.rottentomatoes.com/critic/josefine-a/movies"
get_critics_2 = requests.get(critics_url_2)
critics_soup_2 = BeautifulSoup(get_critics_2.text,"html.parser")
page =critics_soup_2.find('table', {'class': "table table-striped critic-review-table responsive-table"})
movie_url_critics = []
for i in page.find_all("td",{"class":"col-xs-12 col-sm-6 critic-review-table__title-column"}):
    movie_url_critics.append(i.find("a")['href'])
df_5_init = pd.read_html(str(page))[0]
df_5_init['Movie URL'] = movie_url_critics
df_5_init['Critic Name'] = "Josefine A."
df_5_init

In [ ]:
#To scrap all the critics from rotten Tomato
#Takes a long time to scrap all data (Was done in another notebook)
i = 1
while i!= 3658:
    try:
        result = None
        while result is None:
            try:
                result = Critic_scrapper(df_3['URL'][i],df_3['Critic Name'][i])
                df_5_init=df_5_init.append(result)
                i+=1
            except:
                 pass
    except:
        pass

In [ ]:
df_5_init.to_csv("total_critics.csv",sep=";")

In [4]:
df_5_init= pd.read_csv("total_critics.csv",sep=";")

In [396]:
df_5_init.head()

,Unnamed: 0,Rating,T-Meter,Title | Year,Review,Movie URL,Critic Name
0,0,3/5,100%,Charlatan (2020),"Despite the turbulent events of Mikoláek's life, Holland and screenwriter Marek Epstein have produced a remarkably calm film. - One Room With A View EDIT Read More | Posted Mar 9, 2020",https://www.rottentomatoes.com/m/charlatan_2020,Josefine A.
1,1,5/5,100%,Welcome to Chechnya (2020),"[David] France has created a documentary which is both incendiary and vital: we must bear witness to these atrocities and cannot allow them to be forgotten. - One Room With A View EDIT Read More | Posted Mar 6, 2020",https://www.rottentomatoes.com/m/welcome_to_chechnya,Josefine A.
2,2,4/5,No Score Yet,Dry Wind (2020),"Dry Wind's powerful and unabashed representation of homosexuality is momentous, all the more so when viewed against the backdrop of Brazil's current political climate. - One Room With A View EDIT Read More | Posted Mar 5, 2020",https://www.rottentomatoes.com/m/dry_wind,Josefine A.
3,3,2/5,No Score Yet,Delete History (Effacer l'historique) (2020),"Drawn into complete ridicule, there is very little that remains relatable, and the superficial narrative which leads all three to ultimately renounce their digital dependency (at least for a while) fails to move. - One Room With A View EDIT Read More | Posted Mar 2, 2020",https://www.rottentomatoes.com/m/delete_history,Josefine A.
4,4,3/5,45%,Surge (2020),"It's undoubtedly Whishaw who keeps the film compelling with his startling performance, even when the script stumbles somewhat in the final half hour. - One Room With A View EDIT Read More | Posted Mar 2, 2020",https://www.rottentomatoes.com/m/surge,Josefine A.


In [5]:
#Total ok 52862 unique movies
# Group the movies by number of reviews
df_movie_list = df_5_init.groupby(['Title | Year','Movie URL']).size().reset_index(name='Freq')

In [397]:
df_movie_list.head()

,Title | Year,Movie URL,Freq
0,!Women Art Revolution (2011),https://www.rottentomatoes.com/m/women_art_revolution_a_secret_history,13
1,"""L'Origine de la tendresse"" and Other Tales (2008)",https://www.rottentomatoes.com/m/lorigine_de_la_tendresse_and_other_tales,3
2,#AnneFrank. Parallel Stories (Vite parallele) (2019),https://www.rottentomatoes.com/m/annefrank_parallel_stories,4
3,#DigitalLivesMatter (2017),https://www.rottentomatoes.com/m/digitallivesmatter_2017,1
4,#Female Pleasure (2019),https://www.rottentomatoes.com/m/female_pleasure,10


In [6]:
#Select the top1000 movies with most review
top_500_reviewed_movies = df_movie_list.sort_values(by="Freq",ascending = False).head(1000).reset_index(drop=True)

In [399]:
top_500_reviewed_movies.head()

,Title | Year,Movie URL,Freq
0,Joker (2019),https://www.rottentomatoes.com/m/joker_2019,530
1,Once Upon a Time In Hollywood (2019),https://www.rottentomatoes.com/m/once_upon_a_time_in_hollywood,512
2,Us (2019),https://www.rottentomatoes.com/m/us_2019,483
3,Avengers: Endgame (2019),https://www.rottentomatoes.com/m/avengers_endgame,475
4,Captain Marvel (2019),https://www.rottentomatoes.com/m/captain_marvel,466


In [7]:
# init_info_test,init_cast_test = movie_info_fn_cast(top_500_reviewed_movies["Movie URL"][56],top_500_reviewed_movies["Title | Year"][56])

In [191]:
def movie_info_fn_cast(URL,Movie_name):
    dict_formula = {'Rating':0, 'Genre':1, 'Directed By':2, 'Written By':3, 'In Theaters':4, 'On Disc/Streaming':5,'Box Office':6, 'Runtime':7, 'Studio':8, 'Synopsis':9, 'Movie Title':10,"Rating Score":11,"Rating Count":12}
    movie_url = URL
    get_movie = requests.get(movie_url)
    movie_soup = BeautifulSoup(get_movie.text,"html.parser")
    movie_info = movie_soup.find("div",{"class":"panel-body content_body"}).find_all("div")
    synopsis=movie_soup.find_all('div', id="movieSynopsis")[0].string
    synopsis= synopsis.strip().replace("\'","")
    movie_detail_head =movie_soup.find("div",{"class":"panel-body content_body"}).find_all("div",{"class":"meta-label subtle"})
    movie_detail_head_names =  [i.string.replace(":","") for i in movie_detail_head]
    movie_detail_head_names.append("Synopsis")
    movie_detail_head_names.append("Movie Title")
    movie_detail_head_names.append("Rating Score")
    movie_detail_head_names.append("Rating Count")
    movie_detail_values =movie_soup.find("div",{"class":"panel-body content_body"}).find_all("div",{"class":"meta-value"})
    movie_detail_values_val = []
    ratings = movie_soup.find_all("div",{"class":"mop-ratings-wrap__half"})[0]
    for num,i in enumerate(movie_detail_head_names):
        if dict_formula[i] ==0:
            movie_detail_values_val.append(movie_detail_values[num].string.strip())
        if dict_formula[i] ==1:
            movie_detail_values_val.append(movie_detail_values[num].string.replace("\n                        \n                        ","").strip())
        if dict_formula[i] ==2:
            movie_detail_values_val.append([((i.string)+";https://www.rottentomatoes.com"+i['href']) for i in movie_detail_values[num].find_all("a")])
        if dict_formula[i] ==3:
            movie_detail_values_val.append([((i.string)+";https://www.rottentomatoes.com"+i['href']) for i in movie_detail_values[num].find_all("a")])
        if dict_formula[i] ==4:
            movie_detail_values_val.append(movie_detail_values[num].find("time").string.strip())
        if dict_formula[i] ==5:
            movie_detail_values_val.append(movie_detail_values[num].find("time").string.strip())
        if dict_formula[i] ==6:
            movie_detail_values_val.append(movie_detail_values[num].string)
        if dict_formula[i] ==7:
            movie_detail_values_val.append(movie_detail_values[num].find("time").string.strip())
        if dict_formula[i] ==8:
            movie_detail_values_val.append(movie_detail_values[num].string.strip())
        if dict_formula[i] ==9:
            movie_detail_values_val.append(synopsis)
        if dict_formula[i] ==10:
            movie_detail_values_val.append(Movie_name)
        if dict_formula[i] ==11:
            if ratings.find("span",{"class":"mop-ratings-wrap__percentage"}) == None:
                movie_detail_values_val.append("No Data")
            else:
                movie_detail_values_val.append(ratings.find("span",{"class":"mop-ratings-wrap__percentage"}).string.strip())
                
        if dict_formula[i] ==12:
            if ratings.find("small",{"class":"mop-ratings-wrap__text--small"}) == None:
                movie_detail_values_val.append("No Data")
            else:
                movie_detail_values_val.append(ratings.find("small",{"class":"mop-ratings-wrap__text--small"}).string.strip())
                
    tmp_df_2 = pd.DataFrame([movie_detail_values_val],columns=movie_detail_head_names)
    critics_soup = movie_soup
    cast_list = []
    cast_url = []
    init_cast=(critics_soup.find_all("div",{"class":"cast-item media inlineBlock"}))
    for i in range(len(init_cast)):
        cast_list.append(init_cast[i].find("span").string.strip())
        cast_url.append("https://www.rottentomatoes.com"+init_cast[i].find("a")['href'])
    if "cast-item media inlineBlock castRemaining" in str(critics_soup):
        remaining_cast=(critics_soup.find_all("div",{"class":"cast-item media inlineBlock castRemaining"}))
        for i in range(len(remaining_cast)):
            cast_list.append(remaining_cast[i].find("span").string.strip())
            cast_url.append("https://www.rottentomatoes.com"+remaining_cast[i].find("a")['href'])
    _ = {"Cast":cast_list,"URL":cast_url}
    df_cast = pd.DataFrame(_)
    df_cast['Movie Name'] = Movie_name
    return tmp_df_2,df_cast

In [8]:
init_info,init_cast = movie_info_fn_cast(top_500_reviewed_movies["Movie URL"][0],top_500_reviewed_movies["Title | Year"][0])

In [9]:
init_info

,Rating,Genre,Directed By,Written By,In Theaters,On Disc/Streaming,Runtime,Studio,Synopsis,Movie Title,Rating Score,Rating Count
0,"R (for strong bloody violence, disturbing behavior, language and brief sexual images)","Action & Adventure, Drama, Mystery & Suspense",[Todd Phillips;https://www.rottentomatoes.com/celebrity/1045105-todd_phillips],"[Todd Phillips;https://www.rottentomatoes.com/celebrity/1045105-todd_phillips, Scott Silver;https://www.rottentomatoes.com/celebrity/scott_silver]","Oct 4, 2019","Dec 17, 2019",122 minutes,Warner Bros. Pictures,"""Joker"" centers around the iconic arch nemesis and is an original, standalone fictional story not seen before on the big screen. Phillips exploration of Arthur Fleck, who is indelibly portrayed by Joaquin Phoenix, is of a man struggling to find his way in Gothams fracturedsociety. A clown-for-hire by day, he aspires to be a stand-up comic at night...but finds the joke always seems to be on him. Caught in a cyclical existence between apathy and cruelty, Arthur makes one bad decision that brings about a chain reaction of escalating events in this gritty character study.",Joker (2019),68%,555


In [10]:
init_cast

,Cast,URL,Movie Name
0,Joaquin Phoenix,https://www.rottentomatoes.com/celebrity/joaquin_phoenix,Joker (2019)
1,Robert De Niro,https://www.rottentomatoes.com/celebrity/robert_de_niro,Joker (2019)
2,Zazie Beetz,https://www.rottentomatoes.com/celebrity/zazie_beetz,Joker (2019)
3,Bill Camp,https://www.rottentomatoes.com/celebrity/bill-camp,Joker (2019)
4,Frances Conroy,https://www.rottentomatoes.com/celebrity/frances_conroy,Joker (2019)
5,Brett Cullen,https://www.rottentomatoes.com/celebrity/brett_cullen,Joker (2019)
6,Glenn Fleshler,https://www.rottentomatoes.com/celebrity/glenn_fleshler,Joker (2019)
7,Douglas Hodge,https://www.rottentomatoes.com/celebrity/douglas_hodge,Joker (2019)
8,Marc Maron,https://www.rottentomatoes.com/celebrity/marc_maron,Joker (2019)
9,Josh Pais,https://www.rottentomatoes.com/celebrity/josh_pais,Joker (2019)


In [11]:
i=1
while i != len(top_500_reviewed_movies):
    result = []
    while len(result)!=2:
        try:
            info,cast = movie_info_fn_cast(top_500_reviewed_movies["Movie URL"][i],top_500_reviewed_movies["Title | Year"][i]) 
            result = [info,cast]
            init_info=init_info.append(result[0])
            init_cast=init_cast.append(result[1])
            print(i,"Done")
            i+=1
        except:
             pass

1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
20 Done
21 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
38 Done
39 Done
40 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done
49 Done
50 Done
51 Done
52 Done
53 Done
54 Done
55 Done
56 Done
57 Done
58 Done
59 Done
60 Done
61 Done
62 Done
63 Done
64 Done
65 Done
66 Done
67 Done
68 Done
69 Done
70 Done
71 Done
72 Done
73 Done
74 Done
75 Done
76 Done
77 Done
78 Done
79 Done
80 Done
81 Done
82 Done
83 Done
84 Done
85 Done
86 Done
87 Done
88 Done
89 Done
90 Done
91 Done
92 Done
93 Done
94 Done
95 Done
96 Done
97 Done
98 Done
99 Done
100 Done
101 Done
102 Done
103 Done
104 Done
105 Done
106 Done
107 Done
108 Done
109 Done
110 Done
111 Done
112 Done
113 Done
114 Done
115 Done
116 Done
117 Done
118 Done
119 Done
120 Done
121 Done
122 Done
123 Done
1

924 Done
925 Done
926 Done
927 Done
928 Done
929 Done
930 Done
931 Done
932 Done
933 Done
934 Done
935 Done
936 Done
937 Done
938 Done
939 Done
940 Done
941 Done
942 Done
943 Done
944 Done
945 Done
946 Done
947 Done
948 Done
949 Done
950 Done
951 Done
952 Done
953 Done
954 Done
955 Done
956 Done
957 Done
958 Done
959 Done
960 Done
961 Done
962 Done
963 Done
964 Done
965 Done
966 Done
967 Done
968 Done
969 Done
970 Done
971 Done
972 Done
973 Done
974 Done
975 Done
976 Done
977 Done
978 Done
979 Done
980 Done
981 Done
982 Done
983 Done
984 Done
985 Done
986 Done
987 Done
988 Done
989 Done
990 Done
991 Done
992 Done
993 Done
994 Done
995 Done
996 Done
997 Done
998 Done
999 Done


In [400]:
init_info.head()

,Rating,Genre,Directed By,Written By,In Theaters,On Disc/Streaming,Runtime,Studio,Synopsis,Movie Name,Rating Score,Rating Count,Box Office,Movie ID,Genre_IDs
0,"R (for strong bloody violence, disturbing behavior, language and brief sexual images)","Action & Adventure, Drama, Mystery & Suspense",[Todd Phillips;https://www.rottentomatoes.com/celebrity/1045105-todd_phillips],"[Todd Phillips;https://www.rottentomatoes.com/celebrity/1045105-todd_phillips, Scott Silver;https://www.rottentomatoes.com/celebrity/scott_silver]","Oct 4, 2019","Dec 17, 2019",122 minutes,Warner Bros. Pictures,"""Joker"" centers around the iconic arch nemesis and is an original, standalone fictional story not seen before on the big screen. Phillips exploration of Arthur Fleck, who is indelibly portrayed by Joaquin Phoenix, is of a man struggling to find his way in Gothams fracturedsociety. A clown-for-hire by day, he aspires to be a stand-up comic at night...but finds the joke always seems to be on him. Caught in a cyclical existence between apathy and cruelty, Arthur makes one bad decision that brings about a chain reaction of escalating events in this gritty character study.",Joker (2019),68%,555,NaN,M0,"[G0, G1, G2]"
1,"R (for language throughout, some strong graphic violence, drug use, and sexual references)","Comedy, Drama",[Quentin Tarantino;https://www.rottentomatoes.com/celebrity/quentin_tarantino],[Quentin Tarantino;https://www.rottentomatoes.com/celebrity/quentin_tarantino],"Jul 26, 2019","Nov 22, 2019",159 minutes,Columbia Pictures,"Quentin Tarantinos ninth feature film is a story that takes place in Los Angeles in 1969, at the height of hippy Hollywood. The two lead characters are Rick Dalton (Leonardo DiCaprio), former star of a western TV series, and his longtime stunt double Cliff Booth (Brad Pitt). Both are struggling to make it in a Hollywood they dont recognize anymore. But Rick has a very famous next-door neighbor...Sharon Tate.",Once Upon a Time In Hollywood (2019),85%,532,NaN,M1,"[G3, G1]"
2,"R (for violence/terror, and language)","Horror, Mystery & Suspense",[Jordan Peele;https://www.rottentomatoes.com/celebrity/jordan_peele],[Jordan Peele;https://www.rottentomatoes.com/celebrity/jordan_peele],"Mar 22, 2019","Jun 18, 2019",120 minutes,Universal Pictures,"Set in present day along the iconic Northern California coastline, Us, from Monkeypaw Productions, stars Oscar (R) winner Lupita Nyongo as Adelaide Wilson, a woman returning to her beachside childhood home with her husband, Gabe (Black Panthers Winston Duke), and their two children (Shahadi Wright Joseph, Evan Alex) for an idyllic summer getaway. Haunted by an unexplainable and unresolved trauma from her past and compounded by a string of eerie coincidences, Adelaide feels her paranoia elevate to high-alert as she grows increasingly certain that something bad is going to befall her family. After spending a tense beach day with their friends, the Tylers (Emmy winner Elisabeth Moss, Tim Heidecker, Cali Sheldon, Noelle Sheldon), Adelaide and her family return to their vacation home. When darkness falls, the Wilsons discover the silhouette of four figures holding hands as they stand in the driveway. Us pits an endearing American family against a terrifying and uncanny opponent: doppelgängers of themselves.",Us (2019),93%,516,NaN,M2,"[G4, G2]"
3,"PG-13 (for sequences of sci-fi violence and action, and some language)","Action & Adventure, Drama, Science Fiction & Fantasy",[Anthony Russo;https://www.rottentomatoes.com/celebrity/anthony_russo],"[Christopher Markus;https://www.rottentomatoes.com/celebrity/christopher_markus, Stephen McFeely;https://www.rottentomatoes.com/celebrity/stephen_mcfeely]","Apr 26, 2019","Jul 30, 2019",182 minutes,Marvel Studios,"The grave course of events set in motion by Thanos that wiped out half the universe and fractured the Avengers ranks compels the remaining Avengers to take one final stand in Marvel Studios grand conclusion to twenty-two films, ""Avengers: End

In [13]:
x = init_info['Movie Title'].unique()

In [14]:
y= init_cast['Movie Name'].unique()

In [15]:
exception = []
for i in x:
    if i not in y:
        exception.append(i)

In [16]:
# 3 movie has no cast data

In [17]:
init_info.columns

Index(['Rating', 'Genre', 'Directed By', 'Written By', 'In Theaters', 'On Disc/Streaming', 'Runtime', 'Studio', 'Synopsis', 'Movie Title', 'Rating Score', 'Rating Count', 'Box Office'], dtype='object')

In [18]:
init_cast.columns

Index(['Cast', 'URL', 'Movie Name'], dtype='object')

In [19]:
len(init_info["Movie Title"].unique())

1000

In [20]:
df_5_init.columns

Index(['Unnamed: 0', 'Rating', 'T-Meter', 'Title | Year', 'Review', 'Movie URL', 'Critic Name'], dtype='object')

In [21]:
top_1000_critics_list = df_5_init[df_5_init['Title | Year'].isin(x)].drop(['Unnamed: 0'], axis=1).reset_index(drop=True)

In [22]:
top_1000_critics_list.columns

Index(['Rating', 'T-Meter', 'Title | Year', 'Review', 'Movie URL', 'Critic Name'], dtype='object')

In [23]:
#To standardize the column names
top_1000_critics_list.columns= ['Rating Score', 'T-Meter', 'Movie Name', 'Review', 'Movie URL', 'Critic Name']

In [24]:
top_1000_critics_list.columns

Index(['Rating Score', 'T-Meter', 'Movie Name', 'Review', 'Movie URL', 'Critic Name'], dtype='object')

In [25]:
init_info.columns

Index(['Rating', 'Genre', 'Directed By', 'Written By', 'In Theaters', 'On Disc/Streaming', 'Runtime', 'Studio', 'Synopsis', 'Movie Title', 'Rating Score', 'Rating Count', 'Box Office'], dtype='object')

In [26]:
unique_critics = top_1000_critics_list['Critic Name'].unique()

In [27]:
unique_critics

array(['Josefine A.', 'Lysalex Hernández A.', 'Alex Abad-Santos', ...,
       'Guy Webster', 'Princess Weekes', 'James Wegg'], dtype=object)

In [28]:
critics_id = ["C%s"%i for i in range(len(unique_critics))]

In [29]:
data = {
    "Critic Name":unique_critics,
    "Critic ID":critics_id
}
critics_table = pd.DataFrame(data)
critics_table.head(5)

,Critic Name,Critic ID
0,Josefine A.,C0
1,Lysalex Hernández A.,C1
2,Alex Abad-Santos,C2
3,Alana Joli Abbott,C3
4,Harrison Abbott,C4


In [30]:
top_1000_critics_list= top_1000_critics_list.merge(critics_table, on='Critic Name', how='left')

In [31]:
len(critics_id) == len(top_1000_critics_list['Critic ID'].unique())
print("Join Successful!")

Join Successful!


In [32]:
top_1000_critics_list.head(5)

,Rating Score,T-Meter,Movie Name,Review,Movie URL,Critic Name,Critic ID
0,4/5,88%,Shirley (2020),"Shirley unquestionably does its subject justice, both on a personal level and in the wider context of female authorship. - One Room With A View EDIT Read More | Posted Mar 2, 2020",https://www.rottentomatoes.com/m/shirley_2020,Josefine A.,C0
1,5/5,99%,Never Rarely Sometimes Always (2020),"With a narrative that is both universal and deeply personal, Never Rarely Sometimes Always is a film of the utmost urgency, a gut punch of the very best kind. - One Room With A View EDIT Read More | Posted Mar 2, 2020",https://www.rottentomatoes.com/m/never_rarely_sometimes_always,Josefine A.,C0
2,3/5,91%,Bacurau (Nighthawk) (2020),"With a set up as large as this, Bacurau would have benefitted from the more generous space of a (mini-)series in order to explore the nuances of this highly ambitious concept more fully. - One Room With A View EDIT Read More | Posted Jul 8, 2019",https://www.rottentomatoes.com/m/bacurau,Josefine A.,C0
3,4/5,99%,Amazing Grace (2019),"The film is a precious time capsule, preserving the raw and thrilling presence of the Queen of Soul both for those who saw her on stage before, and those who never had the chance. - One Room With A View EDIT Read More | Posted Feb 19, 2019",https://www.rottentomatoes.com/m/amazing_grace_2018,Josefine A.,C0
4,NaN,99%,Apollo 11 (2019),"an excellent opportunity to look at the past and dream about what man or woman can do in the not too distant future. [Full review in Spanish] - La Nación (Costa Rica) EDIT Read More | Posted Jul 29, 2019",https://www.rottentomatoes.com/m/apollo_11_2019,Lysalex Hernández A.,C1


In [33]:
cast_list = (init_cast['Cast'].unique())

In [34]:
cast_id = ["A%s"%i for i in range(len(cast_list))]

In [35]:
data_2 = {
    "Cast":cast_list,
    "Cast ID":cast_id
}
cast_table = pd.DataFrame(data_2)
cast_table.head(5)

,Cast,Cast ID
0,Joaquin Phoenix,A0
1,Robert De Niro,A1
2,Zazie Beetz,A2
3,Bill Camp,A3
4,Frances Conroy,A4


In [36]:
init_cast= init_cast.merge(cast_table, on='Cast', how='left')

In [37]:
len(cast_id) == len(init_cast['Cast ID'].unique())
print("Join Successful!")

Join Successful!


In [38]:
init_cast.head()

,Cast,URL,Movie Name,Cast ID
0,Joaquin Phoenix,https://www.rottentomatoes.com/celebrity/joaquin_phoenix,Joker (2019),A0
1,Robert De Niro,https://www.rottentomatoes.com/celebrity/robert_de_niro,Joker (2019),A1
2,Zazie Beetz,https://www.rottentomatoes.com/celebrity/zazie_beetz,Joker (2019),A2
3,Bill Camp,https://www.rottentomatoes.com/celebrity/bill-camp,Joker (2019),A3
4,Frances Conroy,https://www.rottentomatoes.com/celebrity/frances_conroy,Joker (2019),A4


In [39]:
init_info.columns = ['Rating', 'Genre', 'Directed By', 'Written By', 'In Theaters', 'On Disc/Streaming', 'Runtime', 'Studio', 'Synopsis', 'Movie Name', 'Rating Score', 'Rating Count', 'Box Office']

In [40]:
movie_list = init_info['Movie Name'].unique()

In [41]:
movie_id = ["M%s"%i for i in range(len(movie_list))]

In [42]:
data_3 = {
    "Movie Name":movie_list,
    "Movie ID":movie_id
}
movie_table = pd.DataFrame(data_3)
movie_table.head(5)

,Movie Name,Movie ID
0,Joker (2019),M0
1,Once Upon a Time In Hollywood (2019),M1
2,Us (2019),M2
3,Avengers: Endgame (2019),M3
4,Captain Marvel (2019),M4


In [43]:
init_info= init_info.merge(movie_table, on='Movie Name', how='left')


In [44]:
len(movie_id) == len(init_info['Movie ID'].unique())
print("Join Successful!")

Join Successful!


In [45]:
#Currently we have 3 main table

In [46]:
init_info.head()

,Rating,Genre,Directed By,Written By,In Theaters,On Disc/Streaming,Runtime,Studio,Synopsis,Movie Name,Rating Score,Rating Count,Box Office,Movie ID
0,"R (for strong bloody violence, disturbing behavior, language and brief sexual images)","Action & Adventure, Drama, Mystery & Suspense",[Todd Phillips;https://www.rottentomatoes.com/celebrity/1045105-todd_phillips],"[Todd Phillips;https://www.rottentomatoes.com/celebrity/1045105-todd_phillips, Scott Silver;https://www.rottentomatoes.com/celebrity/scott_silver]","Oct 4, 2019","Dec 17, 2019",122 minutes,Warner Bros. Pictures,"""Joker"" centers around the iconic arch nemesis and is an original, standalone fictional story not seen before on the big screen. Phillips exploration of Arthur Fleck, who is indelibly portrayed by Joaquin Phoenix, is of a man struggling to find his way in Gothams fracturedsociety. A clown-for-hire by day, he aspires to be a stand-up comic at night...but finds the joke always seems to be on him. Caught in a cyclical existence between apathy and cruelty, Arthur makes one bad decision that brings about a chain reaction of escalating events in this gritty character study.",Joker (2019),68%,555,NaN,M0
1,"R (for language throughout, some strong graphic violence, drug use, and sexual references)","Comedy, Drama",[Quentin Tarantino;https://www.rottentomatoes.com/celebrity/quentin_tarantino],[Quentin Tarantino;https://www.rottentomatoes.com/celebrity/quentin_tarantino],"Jul 26, 2019","Nov 22, 2019",159 minutes,Columbia Pictures,"Quentin Tarantinos ninth feature film is a story that takes place in Los Angeles in 1969, at the height of hippy Hollywood. The two lead characters are Rick Dalton (Leonardo DiCaprio), former star of a western TV series, and his longtime stunt double Cliff Booth (Brad Pitt). Both are struggling to make it in a Hollywood they dont recognize anymore. But Rick has a very famous next-door neighbor...Sharon Tate.",Once Upon a Time In Hollywood (2019),85%,532,NaN,M1
2,"R (for violence/terror, and language)","Horror, Mystery & Suspense",[Jordan Peele;https://www.rottentomatoes.com/celebrity/jordan_peele],[Jordan Peele;https://www.rottentomatoes.com/celebrity/jordan_peele],"Mar 22, 2019","Jun 18, 2019",120 minutes,Universal Pictures,"Set in present day along the iconic Northern California coastline, Us, from Monkeypaw Productions, stars Oscar (R) winner Lupita Nyongo as Adelaide Wilson, a woman returning to her beachside childhood home with her husband, Gabe (Black Panthers Winston Duke), and their two children (Shahadi Wright Joseph, Evan Alex) for an idyllic summer getaway. Haunted by an unexplainable and unresolved trauma from her past and compounded by a string of eerie coincidences, Adelaide feels her paranoia elevate to high-alert as she grows increasingly certain that something bad is going to befall her family. After spending a tense beach day with their friends, the Tylers (Emmy winner Elisabeth Moss, Tim Heidecker, Cali Sheldon, Noelle Sheldon), Adelaide and her family return to their vacation home. When darkness falls, the Wilsons discover the silhouette of four figures holding hands as they stand in the driveway. Us pits an endearing American family against a terrifying and uncanny opponent: doppelgängers of themselves.",Us (2019),93%,516,NaN,M2
3,"PG-13 (for sequences of sci-fi violence and action, and some language)","Action & Adventure, Drama, Science Fiction & Fantasy",[Anthony Russo;https://www.rottentomatoes.com/celebrity/anthony_russo],"[Christopher Markus;https://www.rottentomatoes.com/celebrity/christopher_markus, Stephen McFeely;https://www.rottentomatoes.com/celebrity/stephen_mcfeely]","Apr 26, 2019","Jul 30, 2019",182 minutes,Marvel Studios,"The grave course of events set in motion by Thanos that wiped out half the universe and fractured the Avengers ranks compels the remaining Avengers to take one final stand in Marvel Studios grand conclusion to twenty-two films, ""Avengers: Endgame.""",Avengers: Endgame (2019),94%,510,NaN,M

In [47]:
init_cast.head()

,Cast,URL,Movie Name,Cast ID
0,Joaquin Phoenix,https://www.rottentomatoes.com/celebrity/joaquin_phoenix,Joker (2019),A0
1,Robert De Niro,https://www.rottentomatoes.com/celebrity/robert_de_niro,Joker (2019),A1
2,Zazie Beetz,https://www.rottentomatoes.com/celebrity/zazie_beetz,Joker (2019),A2
3,Bill Camp,https://www.rottentomatoes.com/celebrity/bill-camp,Joker (2019),A3
4,Frances Conroy,https://www.rottentomatoes.com/celebrity/frances_conroy,Joker (2019),A4


In [48]:
top_1000_critics_list.head()

,Rating Score,T-Meter,Movie Name,Review,Movie URL,Critic Name,Critic ID
0,4/5,88%,Shirley (2020),"Shirley unquestionably does its subject justice, both on a personal level and in the wider context of female authorship. - One Room With A View EDIT Read More | Posted Mar 2, 2020",https://www.rottentomatoes.com/m/shirley_2020,Josefine A.,C0
1,5/5,99%,Never Rarely Sometimes Always (2020),"With a narrative that is both universal and deeply personal, Never Rarely Sometimes Always is a film of the utmost urgency, a gut punch of the very best kind. - One Room With A View EDIT Read More | Posted Mar 2, 2020",https://www.rottentomatoes.com/m/never_rarely_sometimes_always,Josefine A.,C0
2,3/5,91%,Bacurau (Nighthawk) (2020),"With a set up as large as this, Bacurau would have benefitted from the more generous space of a (mini-)series in order to explore the nuances of this highly ambitious concept more fully. - One Room With A View EDIT Read More | Posted Jul 8, 2019",https://www.rottentomatoes.com/m/bacurau,Josefine A.,C0
3,4/5,99%,Amazing Grace (2019),"The film is a precious time capsule, preserving the raw and thrilling presence of the Queen of Soul both for those who saw her on stage before, and those who never had the chance. - One Room With A View EDIT Read More | Posted Feb 19, 2019",https://www.rottentomatoes.com/m/amazing_grace_2018,Josefine A.,C0
4,NaN,99%,Apollo 11 (2019),"an excellent opportunity to look at the past and dream about what man or woman can do in the not too distant future. [Full review in Spanish] - La Nación (Costa Rica) EDIT Read More | Posted Jul 29, 2019",https://www.rottentomatoes.com/m/apollo_11_2019,Lysalex Hernández A.,C1


In [49]:
Genre_list =[]
for i in init_info["Genre"]:
    _ = (i.split(","))
    for i in _:
        __ = i.strip()
        if __ not in Genre_list:
            Genre_list.append(__)
        

In [50]:
x_ = len(Genre_list)
print("Total of %s Genres"%x_)

Total of 19 Genres


In [57]:
Genre_id = ["G%s"%i for i in range(len(Genre_list))]

In [67]:
_tmp_ = dict(zip(Genre_list,Genre_id))

In [157]:
data_4 = {
    "Genre":Genre_list,
    "Genre ID":Genre_id
}
genre_table = pd.DataFrame(data_4)
genre_table.head(5)

,Genre,Genre ID
0,Action & Adventure,G0
1,Drama,G1
2,Mystery & Suspense,G2
3,Comedy,G3
4,Horror,G4


In [75]:
def gen_Genre_id(x):
    gen_list =[]
    _ = (x.split(","))
    for i in _:
        __ = i.strip()
        gen_list.append(_tmp_[__])
    return gen_list   

In [76]:
init_info["Genre_IDs"] = init_info["Genre"].apply(gen_Genre_id)

In [79]:
init_info.columns

Index(['Rating', 'Genre', 'Directed By', 'Written By', 'In Theaters', 'On Disc/Streaming', 'Runtime', 'Studio', 'Synopsis', 'Movie Name', 'Rating Score', 'Rating Count', 'Box Office', 'Movie ID', 'Genre_IDs'], dtype='object')

In [129]:
main_table = init_info.copy()

In [130]:
main_table = main_table[['Movie ID','Movie Name', 'Genre_IDs','Genre','Studio','Rating Score','Rating Count','Rating' , 'Directed By', 'Written By', 'In Theaters', 'On Disc/Streaming', 'Runtime', 'Synopsis', 'Box Office']]

In [132]:
main_table['Directed By']=main_table['Directed By'].fillna("No Data")

In [138]:
director_list = []
for i in main_table['Directed By']:
    if i !="No Data":
        for j in i:
            __ =(j.split(";")[0])
            if __ not in director_list:
                director_list.append(__)

In [140]:
len(director_list)

615

In [141]:
Director_id = ["D%s"%i for i in range(len(director_list))]

In [142]:
_tmp2_ = dict(zip(director_list,Director_id))

In [152]:
def gen_director_id(x):
    gen_list =[]
    if x == "No Data":
        gen_list.append("No Data")
    else:
        for i in x:
            tmp_d = i.split(";")[0]
            gen_list.append(_tmp2_[tmp_d])
    return gen_list   

In [153]:
main_table['Director ID'] = main_table['Directed By'].apply(gen_director_id)

In [158]:
data_5 = {
    "Director Name":director_list,
    "Director ID":Director_id
}
director_table = pd.DataFrame(data_5)
director_table.head(5)

,Director Name,Director ID
0,Todd Phillips,D0
1,Quentin Tarantino,D1
2,Jordan Peele,D2
3,Anthony Russo,D3
4,Anna Boden,D4


In [160]:
main_table['Written By']=main_table['Written By'].fillna("No Data")

In [161]:
writter_list = []
for i in main_table['Written By']:
    if i !="No Data":
        for j in i:
            __ =(j.split(";")[0])
            if __ not in writter_list:
                writter_list.append(__)

In [162]:
len(writter_list)

1163

In [166]:
writter_id = ["W%s"%i for i in range(len(writter_list))]

In [167]:
_tmp3_ = dict(zip(writter_list,writter_id))

In [169]:
def gen_writter_id(x):
    gen_list =[]
    if x == "No Data":
        gen_list.append("No Data")
    else:
        for i in x:
            tmp_d = i.split(";")[0]
            gen_list.append(_tmp3_[tmp_d])
    return gen_list   

In [171]:
main_table['Writter ID'] = main_table['Written By'].apply(gen_writter_id)

In [172]:
data_6 = {
    "Writter Name":writter_list,
    "Writter ID":writter_id
}
writter_table = pd.DataFrame(data_6)
writter_table.head(5)

,Writter Name,Writter ID
0,Todd Phillips,W0
1,Scott Silver,W1
2,Quentin Tarantino,W2
3,Jordan Peele,W3
4,Christopher Markus,W4


In [194]:
len(main_table["Studio"].unique())

164

In [199]:
main_table[main_table["Studio"].isna()==True]
#There are some movies with no studio DATA

,Movie ID,Movie Name,Genre_IDs,Genre,Studio,Rating Score,Rating Count,Rating,Directed By,Director ID,Written By,Writter ID,In Theaters,On Disc/Streaming,Runtime,Synopsis,Box Office
114,M114,A Wrinkle in Time (2018),"[G0, G1, G7, G5]","Action & Adventure, Drama, Kids & Family, Science Fiction & Fantasy",NaN,42%,324,NR (for thematic elements and some peril),[Ava DuVernay;https://www.rottentomatoes.com/celebrity/ava_duvernay],[D101],No Data,[No Data],"Mar 9, 2018","Jun 5, 2018",109 minutes,"Meg Murry (Storm Reid) is a typical middle school student struggling with issues of self-worth who is desperate to fit in. As the daughter of two world-renowned physicists, she is intelligent and uniquely gifted, as is Megs younger brother Charles Wallace (Deric McCabe), but she has yet to realize it for herself. Making matters even worse is the baffling disappearance of Mr. Murry (Chris Pine), which torments Meg and has left her mother (Gugu Mbatha-Raw) heartbroken. Charles Wallace introduces Meg and her fellow classmate Calvin (Levi Miller) to three celestial guides-Mrs. Which (Oprah Winfrey), Mrs. Whatsit (Reese Witherspoon) and Mrs. Who (Mindy Kaling)-who have journeyed to Earth to help search for their father, and together they set off on their formidable quest. Traveling via a wrinkling of time and space known as tessering, they are soon transported to worlds beyond their imagination where they must confront a powerful evil. To make it back home to Earth, Meg must look deep within herself and embrace her flaws to harness the strength necessary to defeat the darkness closing in on them.",NaN
154,M154,Richard Jewell (2019),[G1],Drama,NaN,76%,274,"R (for language including some sexual references, and brief bloody images)",[Clint Eastwood;https://www.rottentomatoes.com/celebrity/clint_eastwood],[D123],[Billy Ray;https://www.rottentomatoes.com/celebrity/billy_ray],[W149],"Dec 13, 2019","Mar 3, 2020",129 minutes,"Directed by Clint Eastwood and based on true events, ""Richard Jewell"" is a story of what happens when what is reported as fact obscures the truth. ""There is a bomb in Centennial Park. You have thirty minutes."" The world is first introduced to Richard Jewell as the security guard who reports finding the device at the 1996 Atlanta bombing-his report making him a hero whose swift actions save countless lives. But within days, the law enforcement wannabe becomes the FBIs number one suspect, vilified by press and public alike, his life ripped apart. Reaching out to independent, anti-establishment attorney Watson Bryant, Jewell staunchly professes his innocence. But Bryant finds he is out of his depth as he fights the combined powers of the FBI, GBI and APD to clear his clients name, while keeping Richard from trusting the very people trying to destroy him.",NaN
465,M465,Disobedience (2018),"[G1, G9]","Drama, Romance",NaN,84%,200,NR (for some strong sexuality),[Sebastián Lelio;https://www.rottentomatoes.com/celebrity/sebastian_lelio],[D270],"[Sebastián Lelio;https://www.rottentomatoes.com/celebrity/sebastian_lelio, Rebecca Lenkiewicz;https://www.rottentomatoes.com/celebrity/rebecca_lenkiewicz]","[W494, W544]","Apr 27, 2018","Jul 17, 2018",114 minutes,"From Sebastián Lelio, the director of the Academy Award-winning A Fantastic Woman, the film follows a woman as she returns to the community that shunned her decades earlier for an attraction to a childhood friend. Once back, their passions reignite as they explore the boundaries of faith and sexuality. Written by Lelio and Rebecca Lenkiewicz and based on Naomi Aldermans book, the film stars Rachel Weisz, Rachel McAdams and Alessandro Nivola.",NaN
606,M606,Fahrenheit 11/9 (2018),[G13],Documentary,NaN,82%,179,NR (for language and some disturbing material/images),[Michael Moore;https://www.rottentomatoes.com/celebrity/1010918-michael_moore],[D410],[Michael Moore;https://www.rottentomatoes.com/celebrity/1010918-michael_moore],[W773],"Sep 21, 2018","Dec 18, 2018",126 minutes,"Michael Moores ""F

In [200]:
main_table['Studio']=main_table['Studio'].fillna("No Data")

In [207]:
studio_list = main_table['Studio'].unique()

In [208]:
studio_id = ["S%s"%i for i in range(len(studio_list))]

In [209]:
data_7 = {
    "Studio Name":studio_list,
    "Studio ID":studio_id
}
studio_table = pd.DataFrame(data_7)
studio_table.head(5)

,Studio Name,Studio ID
0,Warner Bros. Pictures,S0
1,Columbia Pictures,S1
2,Universal Pictures,S2
3,Marvel Studios,S3
4,Walt Disney Pictures,S4


In [212]:
main_table.columns = ['Movie ID', 'Movie Name', 'Genre_IDs', 'Genre', 'Studio Name', 'Rating Score', 'Rating Count', 'Rating', 'Directed By', 'Director ID', 'Written By','Writter ID' , 'In Theaters', 'On Disc/Streaming', 'Runtime', 'Synopsis', 'Box Office']

In [214]:
main_table= main_table.merge(studio_table, on='Studio Name', how='left')

In [219]:
len(main_table["Studio ID"].unique()) == len(studio_id)
#Join successfully

True

In [220]:
main_table

,Movie ID,Movie Name,Genre_IDs,Genre,Studio Name,Rating Score,Rating Count,Rating,Directed By,Director ID,Written By,Writter ID,In Theaters,On Disc/Streaming,Runtime,Synopsis,Box Office,Studio ID
0,M0,Joker (2019),"[G0, G1, G2]","Action & Adventure, Drama, Mystery & Suspense",Warner Bros. Pictures,68%,555,"R (for strong bloody violence, disturbing behavior, language and brief sexual images)",[Todd Phillips;https://www.rottentomatoes.com/celebrity/1045105-todd_phillips],[D0],"[Todd Phillips;https://www.rottentomatoes.com/celebrity/1045105-todd_phillips, Scott Silver;https://www.rottentomatoes.com/celebrity/scott_silver]","[W0, W1]","Oct 4, 2019","Dec 17, 2019",122 minutes,"""Joker"" centers around the iconic arch nemesis and is an original, standalone fictional story not seen before on the big screen. Phillips exploration of Arthur Fleck, who is indelibly portrayed by Joaquin Phoenix, is of a man struggling to find his way in Gothams fracturedsociety. A clown-for-hire by day, he aspires to be a stand-up comic at night...but finds the joke always seems to be on him. Caught in a cyclical existence between apathy and cruelty, Arthur makes one bad decision that brings about a chain reaction of escalating events in this gritty character study.",NaN,S0
1,M1,Once Upon a Time In Hollywood (2019),"[G3, G1]","Comedy, Drama",Columbia Pictures,85%,532,"R (for language throughout, some strong graphic violence, drug use, and sexual references)",[Quentin Tarantino;https://www.rottentomatoes.com/celebrity/quentin_tarantino],[D1],[Quentin Tarantino;https://www.rottentomatoes.com/celebrity/quentin_tarantino],[W2],"Jul 26, 2019","Nov 22, 2019",159 minutes,"Quentin Tarantinos ninth feature film is a story that takes place in Los Angeles in 1969, at the height of hippy Hollywood. The two lead characters are Rick Dalton (Leonardo DiCaprio), former star of a western TV series, and his longtime stunt double Cliff Booth (Brad Pitt). Both are struggling to make it in a Hollywood they dont recognize anymore. But Rick has a very famous next-door neighbor...Sharon Tate.",NaN,S1
2,M2,Us (2019),"[G4, G2]","Horror, Mystery & Suspense",Universal Pictures,93%,516,"R (for violence/terror, and language)",[Jordan Peele;https://www.rottentomatoes.com/celebrity/jordan_peele],[D2],[Jordan Peele;https://www.rottentomatoes.com/celebrity/jordan_peele],[W3],"Mar 22, 2019","Jun 18, 2019",120 minutes,"Set in present day along the iconic Northern California coastline, Us, from Monkeypaw Productions, stars Oscar (R) winner Lupita Nyongo as Adelaide Wilson, a woman returning to her beachside childhood home with her husband, Gabe (Black Panthers Winston Duke), and their two children (Shahadi Wright Joseph, Evan Alex) for an idyllic summer getaway. Haunted by an unexplainable and unresolved trauma from her past and compounded by a string of eerie coincidences, Adelaide feels her paranoia elevate to high-alert as she grows increasingly certain that something bad is going to befall her family. After spending a tense beach day with their friends, the Tylers (Emmy winner Elisabeth Moss, Tim Heidecker, Cali Sheldon, Noelle Sheldon), Adelaide and her family return to their vacation home. When darkness falls, the Wilsons discover the silhouette of four figures holding hands as they stand in the driveway. Us pits an endearing American family against a terrifying and uncanny opponent: doppelgängers of themselves.",NaN,S2
3,M3,Avengers: Endgame (2019),"[G0, G1, G5]","Action & Adventure, Drama, Science Fiction & Fantasy",Marvel Studios,94%,510,"PG-13 (for sequences of sci-fi violence and action, and some language)",[Anthony Russo;https://www.rottentomatoes.com/celebrity/anthony_russo],[D3],"[Christopher Markus;https://www.rottentomatoes.com/celebrity/christopher_markus, Stephen McFeely;https://www.rottentomatoes.com/celebrity/stephen_mcfeely]","[W4, W5]","Apr 26, 2019","Jul 30, 2019",182 minutes,"The grave course of events set in motion by Thanos that wiped out half the universe and fract

In [236]:
def clean_rating(x):
    return x.split(" ")[0]
main_table["Movie Rating"]= main_table["Rating"].apply(clean_rating)

In [243]:
output_main_table = main_table.drop(['Directed By', 'Written By',"Genre","Studio Name","Synopsis","In Theaters","On Disc/Streaming","Rating"], axis = 1)

In [378]:
output_main_table["Box Office"]=output_main_table["Box Office"].fillna("No Data")

In [384]:
output_main_table.columns = ['Movie ID', 'Movie Name', 'Genre_ID', 'Rating Score', 'Rating Count', 'Director ID', 'Writter ID', 'Runtime', 'Box Office', 'Studio ID', 'Movie Rating']

In [385]:
output_main_table.head()

,Movie ID,Movie Name,Genre_ID,Rating Score,Rating Count,Director ID,Writter ID,Runtime,Box Office,Studio ID,Movie Rating
0,M0,Joker (2019),"[G0, G1, G2]",68%,555,[D0],"[W0, W1]",122 minutes,No Data,S0,R
1,M1,Once Upon a Time In Hollywood (2019),"[G3, G1]",85%,532,[D1],[W2],159 minutes,No Data,S1,R
2,M2,Us (2019),"[G4, G2]",93%,516,[D2],[W3],120 minutes,No Data,S2,R
3,M3,Avengers: Endgame (2019),"[G0, G1, G5]",94%,510,[D3],"[W4, W5]",182 minutes,No Data,S3,PG-13
4,M4,Captain Marvel (2019),"[G0, G5]",78%,505,"[D4, D5]","[W6, W7, W8]",128 minutes,No Data,S3,PG-13


In [302]:
cast_table_final = init_cast.merge(movie_table, on='Movie Name', how='left')

In [303]:
cast_table_final= cast_table_final[["Movie ID","Cast ID","Cast"]]

In [305]:
cast_table_final.columns = ["Movie ID","Cast ID","Cast Name"]

In [295]:
# cast_grouping = pd.DataFrame(init_cast.groupby(["Movie Name"])['Cast ID'].agg(','.join))

In [296]:
# cast_grouping = cast_grouping.merge(movie_table, on='Movie Name', how='left')
# cast_grouping.head()

In [306]:
# cast_grouping = cast_grouping[["Movie ID","Cast ID"]]

In [307]:
# cast_grouping

In [239]:
movie_add_info=main_table[["Movie ID","In Theaters","On Disc/Streaming","Synopsis","Rating"]]

In [241]:
movie_add_info.columns = ["Movie ID","In Theaters","On Disc/Streaming","Synopsis","Rating Description"]

In [245]:
movie_add_info.head()

,Movie ID,In Theaters,On Disc/Streaming,Synopsis,Rating Description
0,M0,"Oct 4, 2019","Dec 17, 2019","""Joker"" centers around the iconic arch nemesis and is an original, standalone fictional story not seen before on the big screen. Phillips exploration of Arthur Fleck, who is indelibly portrayed by Joaquin Phoenix, is of a man struggling to find his way in Gothams fracturedsociety. A clown-for-hire by day, he aspires to be a stand-up comic at night...but finds the joke always seems to be on him. Caught in a cyclical existence between apathy and cruelty, Arthur makes one bad decision that brings about a chain reaction of escalating events in this gritty character study.","R (for strong bloody violence, disturbing behavior, language and brief sexual images)"
1,M1,"Jul 26, 2019","Nov 22, 2019","Quentin Tarantinos ninth feature film is a story that takes place in Los Angeles in 1969, at the height of hippy Hollywood. The two lead characters are Rick Dalton (Leonardo DiCaprio), former star of a western TV series, and his longtime stunt double Cliff Booth (Brad Pitt). Both are struggling to make it in a Hollywood they dont recognize anymore. But Rick has a very famous next-door neighbor...Sharon Tate.","R (for language throughout, some strong graphic violence, drug use, and sexual references)"
2,M2,"Mar 22, 2019","Jun 18, 2019","Set in present day along the iconic Northern California coastline, Us, from Monkeypaw Productions, stars Oscar (R) winner Lupita Nyongo as Adelaide Wilson, a woman returning to her beachside childhood home with her husband, Gabe (Black Panthers Winston Duke), and their two children (Shahadi Wright Joseph, Evan Alex) for an idyllic summer getaway. Haunted by an unexplainable and unresolved trauma from her past and compounded by a string of eerie coincidences, Adelaide feels her paranoia elevate to high-alert as she grows increasingly certain that something bad is going to befall her family. After spending a tense beach day with their friends, the Tylers (Emmy winner Elisabeth Moss, Tim Heidecker, Cali Sheldon, Noelle Sheldon), Adelaide and her family return to their vacation home. When darkness falls, the Wilsons discover the silhouette of four figures holding hands as they stand in the driveway. Us pits an endearing American family against a terrifying and uncanny opponent: doppelgängers of themselves.","R (for violence/terror, and language)"
3,M3,"Apr 26, 2019","Jul 30, 2019","The grave course of events set in motion by Thanos that wiped out half the universe and fractured the Avengers ranks compels the remaining Avengers to take one final stand in Marvel Studios grand conclusion to twenty-two films, ""Avengers: Endgame.""","PG-13 (for sequences of sci-fi violence and action, and some language)"
4,M4,"Mar 8, 2019","Jun 11, 2019","The story follows Carol Danvers as she becomes one of the universes most powerful heroes when Earth is caught in the middle of a galactic war between two alien races. Set in the 1990s, Captain Marvel is an all-new adventure from a previously unseen period in the history of the Marvel Cinematic Universe.","PG-13 (for sequences of sci-fi violence and action, and brief suggestive language)"


In [361]:
critics_review_table = top_1000_critics_list.copy()

In [362]:
critics_review_table= critics_review_table.merge(movie_table, on='Movie Name', how='left')

In [363]:
critics_review_table.columns

Index(['Rating Score', 'T-Meter', 'Movie Name', 'Review', 'Movie URL', 'Critic Name', 'Critic ID', 'Movie ID'], dtype='object')

In [364]:
critics_review_table = critics_review_table[['Movie ID','Critic ID','Rating Score', 'T-Meter', 'Review']]

In [365]:
critics_review_table.columns=['Movie ID','Critic ID','5 points score', 'Rating Score', 'Review']

In [366]:
critics_review_table

,Movie ID,Critic ID,5 points score,Rating Score,Review
0,M596,C0,4/5,88%,"Shirley unquestionably does its subject justice, both on a personal level and in the wider context of female authorship. - One Room With A View EDIT Read More | Posted Mar 2, 2020"
1,M552,C0,5/5,99%,"With a narrative that is both universal and deeply personal, Never Rarely Sometimes Always is a film of the utmost urgency, a gut punch of the very best kind. - One Room With A View EDIT Read More | Posted Mar 2, 2020"
2,M880,C0,3/5,91%,"With a set up as large as this, Bacurau would have benefitted from the more generous space of a (mini-)series in order to explore the nuances of this highly ambitious concept more fully. - One Room With A View EDIT Read More | Posted Jul 8, 2019"
3,M874,C0,4/5,99%,"The film is a precious time capsule, preserving the raw and thrilling presence of the Queen of Soul both for those who saw her on stage before, and those who never had the chance. - One Room With A View EDIT Read More | Posted Feb 19, 2019"
4,M536,C1,NaN,99%,"an excellent opportunity to look at the past and dream about what man or woman can do in the not too distant future. [Full review in Spanish] - La Nación (Costa Rica) EDIT Read More | Posted Jul 29, 2019"
...,...,...,...,...,...
198296,M853,C2527,4/5,79%,"The whole truth and mostly just the truth - JWR EDIT Read More | Posted Dec 31, 2009"
198297,M675,C2527,3/5,91%,"Much splat, little art - JWR EDIT Read More | Posted Jul 23, 2009"
198298,M854,C2527,3/5,98%,"The best thing about director Darren Aronofsky's realization of Robert Siegel's original screenplay is Mickey Rourke's portrayal of Randy ""the Ram"" Robinson. - JWR EDIT Read More | Posted Feb 24, 2009"
198299,M509,C2527,5/5,91%,"... this production could well be billed as the most entertaining film of 2008. - JWR EDIT Read More | Posted Jan 28, 2009"


In [374]:
def clean_review(x):
    tmp_a = x.split("-")[0]
    return tmp_a
def clean_review_2(x):
    tmp_b = x.split("-")[-1].split("Posted ")[-1]
    return tmp_b


In [367]:
critics_review_table["Review_2"] = critics_review_table["Review"].apply(clean_review)
critics_review_table["Date"] = critics_review_table["Review"].apply(clean_review_2)

In [370]:
critics_review_table = critics_review_table.drop(['Review'], axis=1)

In [373]:
critics_review_table.columns = ['Movie ID', 'Critic ID', '5 points score', 'Rating Score', 'Review', 'Date']

In [375]:
critics_review_table.head()

,Movie ID,Critic ID,5 points score,Rating Score,Review,Date
0,M596,C0,4/5,88%,"Shirley unquestionably does its subject justice, both on a personal level and in the wider context of female authorship.","Mar 2, 2020"
1,M552,C0,5/5,99%,"With a narrative that is both universal and deeply personal, Never Rarely Sometimes Always is a film of the utmost urgency, a gut punch of the very best kind.","Mar 2, 2020"
2,M880,C0,3/5,91%,"With a set up as large as this, Bacurau would have benefitted from the more generous space of a (mini","Jul 8, 2019"
3,M874,C0,4/5,99%,"The film is a precious time capsule, preserving the raw and thrilling presence of the Queen of Soul both for those who saw her on stage before, and those who never had the chance.","Feb 19, 2019"
4,M536,C1,NaN,99%,an excellent opportunity to look at the past and dream about what man or woman can do in the not too distant future. [Full review in Spanish],"Jul 29, 2019"


# Here are all of the table

In [386]:
# First table is the movie list & info table
output_main_table.head()

,Movie ID,Movie Name,Genre_ID,Rating Score,Rating Count,Director ID,Writter ID,Runtime,Box Office,Studio ID,Movie Rating
0,M0,Joker (2019),"[G0, G1, G2]",68%,555,[D0],"[W0, W1]",122 minutes,No Data,S0,R
1,M1,Once Upon a Time In Hollywood (2019),"[G3, G1]",85%,532,[D1],[W2],159 minutes,No Data,S1,R
2,M2,Us (2019),"[G4, G2]",93%,516,[D2],[W3],120 minutes,No Data,S2,R
3,M3,Avengers: Endgame (2019),"[G0, G1, G5]",94%,510,[D3],"[W4, W5]",182 minutes,No Data,S3,PG-13
4,M4,Captain Marvel (2019),"[G0, G5]",78%,505,"[D4, D5]","[W6, W7, W8]",128 minutes,No Data,S3,PG-13


In [248]:
# Second table is additional movie info table
movie_add_info.head()

,Movie ID,In Theaters,On Disc/Streaming,Synopsis,Rating Description
0,M0,"Oct 4, 2019","Dec 17, 2019","""Joker"" centers around the iconic arch nemesis and is an original, standalone fictional story not seen before on the big screen. Phillips exploration of Arthur Fleck, who is indelibly portrayed by Joaquin Phoenix, is of a man struggling to find his way in Gothams fracturedsociety. A clown-for-hire by day, he aspires to be a stand-up comic at night...but finds the joke always seems to be on him. Caught in a cyclical existence between apathy and cruelty, Arthur makes one bad decision that brings about a chain reaction of escalating events in this gritty character study.","R (for strong bloody violence, disturbing behavior, language and brief sexual images)"
1,M1,"Jul 26, 2019","Nov 22, 2019","Quentin Tarantinos ninth feature film is a story that takes place in Los Angeles in 1969, at the height of hippy Hollywood. The two lead characters are Rick Dalton (Leonardo DiCaprio), former star of a western TV series, and his longtime stunt double Cliff Booth (Brad Pitt). Both are struggling to make it in a Hollywood they dont recognize anymore. But Rick has a very famous next-door neighbor...Sharon Tate.","R (for language throughout, some strong graphic violence, drug use, and sexual references)"
2,M2,"Mar 22, 2019","Jun 18, 2019","Set in present day along the iconic Northern California coastline, Us, from Monkeypaw Productions, stars Oscar (R) winner Lupita Nyongo as Adelaide Wilson, a woman returning to her beachside childhood home with her husband, Gabe (Black Panthers Winston Duke), and their two children (Shahadi Wright Joseph, Evan Alex) for an idyllic summer getaway. Haunted by an unexplainable and unresolved trauma from her past and compounded by a string of eerie coincidences, Adelaide feels her paranoia elevate to high-alert as she grows increasingly certain that something bad is going to befall her family. After spending a tense beach day with their friends, the Tylers (Emmy winner Elisabeth Moss, Tim Heidecker, Cali Sheldon, Noelle Sheldon), Adelaide and her family return to their vacation home. When darkness falls, the Wilsons discover the silhouette of four figures holding hands as they stand in the driveway. Us pits an endearing American family against a terrifying and uncanny opponent: doppelgängers of themselves.","R (for violence/terror, and language)"
3,M3,"Apr 26, 2019","Jul 30, 2019","The grave course of events set in motion by Thanos that wiped out half the universe and fractured the Avengers ranks compels the remaining Avengers to take one final stand in Marvel Studios grand conclusion to twenty-two films, ""Avengers: Endgame.""","PG-13 (for sequences of sci-fi violence and action, and some language)"
4,M4,"Mar 8, 2019","Jun 11, 2019","The story follows Carol Danvers as she becomes one of the universes most powerful heroes when Earth is caught in the middle of a galactic war between two alien races. Set in the 1990s, Captain Marvel is an all-new adventure from a previously unseen period in the history of the Marvel Cinematic Universe.","PG-13 (for sequences of sci-fi violence and action, and brief suggestive language)"


In [254]:
# Third table is additional studio table
studio_table.head()

,Studio Name,Studio ID
0,Warner Bros. Pictures,S0
1,Columbia Pictures,S1
2,Universal Pictures,S2
3,Marvel Studios,S3
4,Walt Disney Pictures,S4


In [252]:
# 4th table is the writer table
writter_table.head()

,Writter Name,Writter ID
0,Todd Phillips,W0
1,Scott Silver,W1
2,Quentin Tarantino,W2
3,Jordan Peele,W3
4,Christopher Markus,W4


In [257]:
# 5th table is the director table
director_table.head()

,Director Name,Director ID
0,Todd Phillips,D0
1,Quentin Tarantino,D1
2,Jordan Peele,D2
3,Anthony Russo,D3
4,Anna Boden,D4


In [259]:
# 6th table is the genre table
genre_table.head()

,Genre,Genre ID
0,Action & Adventure,G0
1,Drama,G1
2,Mystery & Suspense,G2
3,Comedy,G3
4,Horror,G4


In [308]:
# 7th table is the cast table
cast_table_final.head()

,Movie ID,Cast ID,Cast Name
0,M0,A0,Joaquin Phoenix
1,M0,A1,Robert De Niro
2,M0,A2,Zazie Beetz
3,M0,A3,Bill Camp
4,M0,A4,Frances Conroy


In [263]:
# 8th table is the critics table
critics_table.head()

,Critic Name,Critic ID
0,Josefine A.,C0
1,Lysalex Hernández A.,C1
2,Alex Abad-Santos,C2
3,Alana Joli Abbott,C3
4,Harrison Abbott,C4


In [376]:
# 9th table is the critics review table
critics_review_table.head()

,Movie ID,Critic ID,5 points score,Rating Score,Review,Date
0,M596,C0,4/5,88%,"Shirley unquestionably does its subject justice, both on a personal level and in the wider context of female authorship.","Mar 2, 2020"
1,M552,C0,5/5,99%,"With a narrative that is both universal and deeply personal, Never Rarely Sometimes Always is a film of the utmost urgency, a gut punch of the very best kind.","Mar 2, 2020"
2,M880,C0,3/5,91%,"With a set up as large as this, Bacurau would have benefitted from the more generous space of a (mini","Jul 8, 2019"
3,M874,C0,4/5,99%,"The film is a precious time capsule, preserving the raw and thrilling presence of the Queen of Soul both for those who saw her on stage before, and those who never had the chance.","Feb 19, 2019"
4,M536,C1,NaN,99%,an excellent opportunity to look at the past and dream about what man or woman can do in the not too distant future. [Full review in Spanish],"Jul 29, 2019"
